In [ ]:
import numpy as np
import matplotlib as mpl
import copy as copy
import sys
sys.path.append("../../TychePlot/")
from LifetimePlot import LifetimePlot
from Filereader import comma_str_to_float
from Plot_processing import calc_plot_list
mpl.use("pgf")

In [ ]:
name="TFSi"
spectra_subdir="./lt/50ns/" # formerly input_file_path
output_file_path="lt_plot/"
inputfileList=[
    "MCP35/native_50ns_MCP35_1000.dac",
    "MCP15/LiTFSi_1__1000.dac",
    "MCP15/LiTFSi_10__1000.dac",
    "MCP35/LiTFSi_100__1000.dac",
    "MCP15/KTFSi_0_1__1000_1.dac",
    "MCP15/KTFSi_1__1000.dac",
    "MCP15/KTFSi_10__1000.dac",
]
labelList=[
    "native",
    "LiTFSi 1\,wt.\,\%",
    "LiTFSi 10\,wt.\,\%",
    "LiTFSi 100\,wt.\,\%",
    "KTFSi 0.1\,wt.\,\%",
    "KTFSi 1\,wt.\,\%",
    "KTFSi 10\,wt.\,\%",
]
locs=[511]*len(labelList)

fileformat=dict(index_col=0, header=0 ,sep="\t", encoding="utf-8", converters={0:comma_str_to_float})

Time=1
Intensity=2
fill="_"

parse_data_style="streak"
multithreaded=True

In [ ]:
## figure
figscale=0.49
present_width_pt=665.79451
present_HW_ratio=1#0.82

present={
            #"customFontsize":[3,3,1.5,2,2],
            "fig_width_pt":present_width_pt*figscale,
            "scaleX":(present_width_pt*figscale)/LifetimePlot.fig_width_default_pt,
            "fixedFigWidth":True,
            "HWratio":present_HW_ratio,
            "titleBool":False,
            "showColLabel": ["","","","", "", "","","","", ""],
            #"legendBool":False,
            "filenamePrefix":output_file_path+"presentation/",
            "labelPad":2.5,
            "axRect":[0.175,0.175,0.65,0.65]
}

In [ ]:
inputParameters={
                 "fileFormat":fileformat,
                 "labels":labelList,
                 "titleBool":False,
                 "useTex":True,
                 "append_col_in_label":False,
                 "filenamePrefix":output_file_path,
                 "markerSize":1,
                 "showMarkers":False,
                 "iterMarkers":False,
                 "showLines":True,
                 "iterBoth":False,
                 "saveProps":{"saveAs":".png", "dpi":600},
                # Lt Props
                 "normalize_peak":False,
                 "set_peak_to_zero":True,
                 "parse_data_style":"streak",
                 "locs":locs
}

inputParameters_fit={
                 "fileFormat":fileformat,
                 "labels":labelList,
                 "titleBool":False,
                 "useTex":True,
                 "append_col_in_label":False,
                 "filenamePrefix":output_file_path,
                 "markerSize":1,
                 "showMarkers":True,
                 "iterMarkers":True,
                 "showLines":False,
                 "iterBoth":False,
                 "saveProps":{"saveAs":".png", "dpi":600},
                # Lt Props
                 "normalize_peak":False,
                 "set_peak_to_zero":True,
                 "parse_data_style":"streak",
                 "locs":locs
}

optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in s
                                    None,#None
                                    None,#Time
                                    None,#Intensity

                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#Time
                                    [10**-3,2],#Intensity
                                  ]
}

desiredPlots=[
                {
                    "xCol":Time,
                    "yCol":Intensity,
                }, ]

desiredPlots_fit_double=[
                {
                    "xCol":Time,
                    "yCol":Intensity,
                    "custom":{
                        "fitList":[
                ([0,40],[0,40],LifetimePlot.doubleExp,(6,0.5,10**-3, 25, 0.5),(25,0.02),None,{"bounds":([0,0,0,0,0],[100,1,10**-2,5000,1]),"maxfev":100000}),
            ],
                    }
                }, ]
desiredPlots_fit_triple=[
                {
                    "xCol":Time,
                    "yCol":Intensity,
                    "custom":{
                        "fitList":[
                ([0,50],[0,50],LifetimePlot.tripleExp,(3.4,0.95,10**-3,53,0.025, 300, 0.025),(20,0.1),None,{"bounds":([0,0,0,0,0,0,0],[100,1,10**-2,5000,1,5000,1]),"maxfev":100000}),
            ],
                    }
                }, ]

In [ ]:
groups=[
        (0,1,2,3,),
        (0,4,5,6),
        (0,),
        (1,),
        (2,),
        (3,),
        (4,),
        (5,),
        (6,),
]
group_names=[
                "LiTFSi",
                "KTFSi",
                "native",
                "LiTFSi_1wt",
                "LiTFSi_10wt",
                "LiTFSi_100wt",
                "KTFSi_0_1wt",
                "KTFSi_1wt",
                "KTFSi_10wt",
]
group_specific_input=[
                        {
                            "desiredPlots":desiredPlots,
                            "inputParameters":inputParameters,
                            "optionalParameters":optionalParameters,
                        },
]*2+[
                        {
                            "desiredPlots":desiredPlots_fit_triple,
                            "inputParameters":inputParameters_fit,
                            "optionalParameters":optionalParameters,
                        },
]*(len(groups)-2)


inputfileList=[[spectra_subdir+inputFile] for inputFile in inputfileList]

In [ ]:
plot_prop_list=[]
feature="both"
for group,group_name,specific_input in zip(groups,group_names,group_specific_input):
    fileList=[]
    labels_loc=[]
    try:
        inputParameters_loc=copy.deepcopy(specific_input["inputParameters"])
    except:
        inputParameters_loc=copy.deepcopy(inputParameters)
    try:
        optionalParameters_loc=copy.deepcopy(specific_input["optionalParameters"])
    except:
        optionalParameters_loc=copy.deepcopy(optionalParameters)
    try:
        desiredPlots_loc=copy.deepcopy(specific_input["desiredPlots"])
    except:
        desiredPlots_loc=copy.deepcopy(desiredPlots)
    for pos in group:
        if isinstance(pos, tuple):
            fileList.append(inputfileList[pos[0]])
            labels_loc.append(labelList[pos[1]])
        else:
            fileList.append(inputfileList[pos])
            labels_loc.append(labelList[pos])
    inputParameters_loc["labels"]=labels_loc
    plot_prop_list.append(
        {
             "name":name+fill+group_name,
             "fileList":fileList,
             "desiredPlots":desiredPlots_loc,
             "present_params":present,
             "inputParameters":inputParameters_loc,
             "plot_class":LifetimePlot,
             "optionalParameters":optionalParameters_loc,
             "feature":feature,
             "multithreaded":multithreaded
        }
        )
results=calc_plot_list(plot_prop_list, multithreaded=multithreaded)

In [ ]:
fitter=[]
for result in results[2:]:
    fitter.append(result[0][0].fitterList[0].params)

In [ ]:
#tau, amp, offset, tau2, amp2, tau3, amp3
fitter_array=np.asarray(fitter)
tau1=fitter_array[:,0]
tau2=fitter_array[:,3]
tau3=fitter_array[:,5]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
all_rates = pd.DataFrame(zip(group_names[2:]*2, ["k$_1$"]*len(tau1)+["k$_2$"]*len(tau2), list(1/tau1)+list(1/tau2)), columns=["sample", "Decay type", "Decay rate (1/ns)"])
only_li_rates = pd.DataFrame(zip(group_names[2:6]*2, ["k$_1$"]*len(tau1[:4])+["k$_2$"]*len(tau2[:4]), list(1/tau1[:4])+list(1/tau2[:4])), columns=["sample", "Decay type", "Decay rate (1/ns)"])
only_k_rates = pd.DataFrame(zip(([group_names[2]]+group_names[6:])*2, ["k$_1$"]*len(tau1[3:])+["k$_2$"]*len(tau2[3:]), [1/tau1[0]]+list(1/tau1[4:])+[1/tau2[0]]+list(1/tau2[4:])), columns=["sample", "Decay type", "Decay rate (1/ns)"])

In [ ]:
fig, ax = LifetimePlot.newFig()
sns.barplot(x="Decay type", y="Decay rate (1/ns)", hue="sample", data=only_li_rates, palette=LifetimePlot.default_colors[:4], ax=ax)
fig.savefig("li_rates.pdf")

In [ ]:
fig, ax = LifetimePlot.newFig()
sns.barplot(x="Decay type", y="Decay rate (1/ns)", hue="sample", data=only_k_rates, palette=[LifetimePlot.default_colors[0]]+LifetimePlot.default_colors[4:], ax=ax)
fig.savefig("k_rates.pdf")

In [ ]:
fig, ax = LifetimePlot.newFig()
sns.barplot(x="Decay type", y="Decay rate (1/ns)", hue="sample", data=all_rates, palette=LifetimePlot.default_colors, ax=ax)
fig.savefig("all_rates.pdf")